In [38]:
!apt install swig
!pip install "stable-baselines3[extra]>=2.0.0a4"

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.



In [39]:
import gymnasium as gym
from stable_baselines3 import A2C, SAC, PPO, TD3
import numpy as np

from stable_baselines3.common.evaluation import evaluate_policy

In [40]:
eval_env = gym.make("Pendulum-v1")

In [41]:
default_model = SAC(
    "MlpPolicy",
    "Pendulum-v1",
    verbose=1,
    seed=0,
    batch_size=64,
    policy_kwargs=dict(net_arch=[64, 64]),
).learn(8000)

Using cpu device
Creating environment from the given name 'Pendulum-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.38e+03 |
| time/              |           |
|    episodes        | 4         |
|    fps             | 503       |
|    time_elapsed    | 1         |
|    total_timesteps | 800       |
| train/             |           |
|    actor_loss      | 18.7      |
|    critic_loss     | 1.46      |
|    ent_coef        | 0.811     |
|    ent_coef_loss   | -0.346    |
|    learning_rate   | 0.0003    |
|    n_updates       | 699       |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.45e+03 |
| time/              |           |
|    episodes        | 8         |
|    fps             | 466       |
|    time_e

In [42]:
mean_reward, std_reward = evaluate_policy(default_model, eval_env, n_eval_episodes=100)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

/opt/homebrew/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:-395.15 +/- 101.77


In [43]:
tuned_model = SAC(
    "MlpPolicy",
    "Pendulum-v1",
    batch_size=256,
    verbose=1,
    policy_kwargs=dict(net_arch=[256, 256]),
    seed=0,
).learn(8000)

Using cpu device
Creating environment from the given name 'Pendulum-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.37e+03 |
| time/              |           |
|    episodes        | 4         |
|    fps             | 248       |
|    time_elapsed    | 3         |
|    total_timesteps | 800       |
| train/             |           |
|    actor_loss      | 22.4      |
|    critic_loss     | 0.272     |
|    ent_coef        | 0.812     |
|    ent_coef_loss   | -0.341    |
|    learning_rate   | 0.0003    |
|    n_updates       | 699       |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.39e+03 |
| time/              |           |
|    episodes        | 8         |
|    fps             | 218       |
|    time_e

In [44]:
mean_reward, std_reward = evaluate_policy(tuned_model, eval_env, n_eval_episodes=100)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:-152.42 +/- 84.35


In [45]:
from stable_baselines3.common.callbacks import BaseCallback

In [46]:
class SimpleCallback(BaseCallback):
    def __init__(self, verbose: int = 0):
        super(SimpleCallback,self).__init__(verbose)
        self._called = False

    def _on_step(self):
        if not self._called:
            print("callback - first call")
            self._called = True
            return True
        print("callback - second call")
        return False

In [47]:
model = SAC("MlpPolicy", "Pendulum-v1", verbose=1)
model.learn(8000, callback=SimpleCallback())

Using cpu device
Creating environment from the given name 'Pendulum-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
callback - first call
callback - second call


In [48]:
import os

import numpy as np

from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.results_plotter import load_results, ts2xy

In [49]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    def __init__(self, check_freq, log_dir, verbose: int = 1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir,"best_model")
        self.best_mean_reward = -np.inf
    
    def _init_callback(self) -> None:
        if self.save_path is not None:
            os.makedirs(self.save_path,exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            x, y = ts2xy(load_results(self.log_dir),"timesteps")
            if len(x)>0:
                mean_reward = np.mean(y[-100:])
                if self.verbose > 0: 
                    print("Num timesteps: {}".format(self.num_timesteps))
                    print(
                        "Best mean reward: {:.2f} - Last mean reward per episode: {:.2f}".format(
                            self.best_mean_reward, mean_reward
                        )
                    )
                
                if mean_reward > self.best_mean_reward:
                    self.best_mean_reward = mean_reward
                    if self.verbose > 0 :
                        print("Saving new best model at {} timesteps".format(x[-1]))
                        print("Saving new best model to {}.zip".format(self.save_path))
                    self.model.save(self.save_path)
        return True
                    

In [50]:
log_dir = "/tmp/gym/"
os.makedirs(log_dir,exist_ok=True)

env = make_vec_env("CartPole-v1",n_envs=1,monitor_dir = log_dir)

callback = SaveOnBestTrainingRewardCallback(check_freq=20, log_dir=log_dir, verbose=1)

model = A2C("MlpPolicy", env, verbose=0)
model.learn(total_timesteps=5000, callback=callback)

Num timesteps: 40
Best mean reward: -inf - Last mean reward per episode: 37.00
Saving new best model at 37 timesteps
Saving new best model to /tmp/gym/best_model.zip
Num timesteps: 60
Best mean reward: 37.00 - Last mean reward per episode: 25.00
Num timesteps: 80
Best mean reward: 37.00 - Last mean reward per episode: 23.67
Num timesteps: 100
Best mean reward: 37.00 - Last mean reward per episode: 23.67
Num timesteps: 120
Best mean reward: 37.00 - Last mean reward per episode: 23.20
Num timesteps: 140
Best mean reward: 37.00 - Last mean reward per episode: 23.20
Num timesteps: 160
Best mean reward: 37.00 - Last mean reward per episode: 22.86
Num timesteps: 180
Best mean reward: 37.00 - Last mean reward per episode: 22.86
Num timesteps: 200
Best mean reward: 37.00 - Last mean reward per episode: 22.86
Num timesteps: 220
Best mean reward: 37.00 - Last mean reward per episode: 26.62
Num timesteps: 240
Best mean reward: 37.00 - Last mean reward per episode: 26.62
Num timesteps: 260
Best me

In [51]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook


class PlottingCallback(BaseCallback):
    """
    Callback for plotting the performance in realtime.

    :param verbose: (int)
    """
    def __init__(self, verbose=1):
        super().__init__(verbose)
        self._plot = None

    def _on_step(self) -> bool:
        # get the monitor's data
        x, y = ts2xy(load_results(log_dir), 'timesteps')
        if self._plot is None: # make the plot
            plt.ion()
            fig = plt.figure(figsize=(6,3))
            ax = fig.add_subplot(111)
            line, = ax.plot(x, y)
            self._plot = (line, ax, fig)
            plt.show()
        else: # update and rescale the plot
            self._plot[0].set_data(x, y)
            self._plot[-2].relim()
            self._plot[-2].set_xlim([self.locals["total_timesteps"] * -0.02, 
                                    self.locals["total_timesteps"] * 1.02])
            self._plot[-2].autoscale_view(True,True,True)
            self._plot[-1].canvas.draw()
        
# Create log dir
log_dir = "/tmp/gym/"
os.makedirs(log_dir, exist_ok=True)

# Create and wrap the environment
env = make_vec_env('MountainCarContinuous-v0', n_envs=1, monitor_dir=log_dir)

plotting_callback = PlottingCallback()
        
model = PPO('MlpPolicy', env, verbose=0)
model.learn(10000, callback=plotting_callback)

<IPython.core.display.Javascript object>

In [52]:
from tqdm.auto import tqdm


class ProgressBarCallback(BaseCallback):
    """
    :param pbar: (tqdm.pbar) Progress bar object
    """

    def __init__(self, pbar):
        super().__init__()
        self._pbar = pbar

    def _on_step(self):
        # Update the progress bar:
        self._pbar.n = self.num_timesteps
        self._pbar.update(0)


# this callback uses the 'with' block, allowing for correct initialisation and destruction
class ProgressBarManager(object):
    def __init__(self, total_timesteps):  # init object with total timesteps
        self.pbar = None
        self.total_timesteps = total_timesteps

    def __enter__(self):  # create the progress bar and callback, return the callback
        self.pbar = tqdm(total=self.total_timesteps)

        return ProgressBarCallback(self.pbar)

    def __exit__(self, exc_type, exc_val, exc_tb):  # close the callback
        self.pbar.n = self.total_timesteps
        self.pbar.update(0)
        self.pbar.close()


model = TD3("MlpPolicy", "Pendulum-v1", verbose=0)
# Using a context manager garanties that the tqdm progress bar closes correctly
with ProgressBarManager(2000) as callback:
    model.learn(2000, callback=callback)

100%|██████████| 2000/2000 [00:05<00:00, 365.03it/s]


In [53]:
from stable_baselines3.common.callbacks import CallbackList

# Create log dir
log_dir = "/tmp/gym/"
os.makedirs(log_dir, exist_ok=True)

# Create and wrap the environment
env = make_vec_env('CartPole-v1', n_envs=1, monitor_dir=log_dir)

# Create callbacks
auto_save_callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=log_dir)

model = PPO('MlpPolicy', env, verbose=0)
with ProgressBarManager(1000) as progress_callback:
  # This is equivalent to callback=CallbackList([progress_callback, auto_save_callback])
  model.learn(1000, callback=[progress_callback, auto_save_callback])

1590it [00:00, 7875.09it/s]                         

Num timesteps: 1000
Best mean reward: -inf - Last mean reward per episode: 21.41
Saving new best model at 985 timesteps
Saving new best model to /tmp/gym/best_model.zip
Num timesteps: 2000
Best mean reward: 21.41 - Last mean reward per episode: 21.98
Saving new best model at 2000 timesteps
Saving new best model to /tmp/gym/best_model.zip


100%|██████████| 1000/1000 [00:00<00:00, 1836.62it/s]


In [84]:


import sys


class EvalCallback(BaseCallback):
    """
    Callback for evaluating an agent.

    :param eval_env: (gym.Env) The environment used for initialization
    :param n_eval_episodes: (int) The number of episodes to test the agent
    :param eval_freq: (int) Evaluate the agent every eval_freq call of the callback.
    """

    def __init__(self, eval_env, n_eval_episodes=5, eval_freq=20):
        super().__init__()
        self.eval_env = eval_env
        self.n_eval_episodes = n_eval_episodes
        self.eval_freq = eval_freq
        self.best_mean_reward = -np.inf

    def _on_step(self):
        """
        This method will be called by the model.

        :return: (bool)
        """

        # self.n_calls is automatically updated because
        # we derive from BaseCallback
        if self.n_calls % self.eval_freq == 0:
            # === YOUR CODE HERE ===#
            # Evaluate the agent:
            # you need to do self.n_eval_episodes loop using self.eval_env
            # hint: you can use self.model.predict(obs, deterministic=True)
            
            
            for _ in range(self.n_eval_episodes):
                obs = self.eval_env.reset()[0]
                done = False
                total_ep_reward = 0
                while not done: 
                    action, a = self.model.predict(obs,deterministic=True)
                    obs,reward,done, *_ = self.eval_env.step(action)
                    total_ep_reward += reward
                
                if total_ep_reward>self.best_mean_reward:
                    self.best_mean_reward = total_ep_reward
                    self.model.save("best_model") 
                    print("Best mean reward: {:.2f}".format(self.best_mean_reward))
            # Save the agent if needed
            # and update self.best_mean_reward

            

            # ====================== #
        return True

In [85]:
# Env used for training
env = gym.make("CartPole-v1")
# Env for evaluating the agent
eval_env = gym.make("CartPole-v1")

# === YOUR CODE HERE ===#
# Create the callback object
callback = EvalCallback(eval_env=eval_env)

# Create the RL model


model = A2C(
    "MlpPolicy",
    env=env,
    verbose=1,
    seed=0,
)

# ====================== #

# Train the RL model
model.learn(int(100_000), callback=callback)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Best mean reward: 328.00
Best mean reward: 332.00
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 47       |
|    ep_rew_mean        | 47       |
| time/                 |          |
|    fps                | 565      |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.596   |
|    explained_variance | -0.0575  |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 1.09     |
|    value_loss         | 9.74     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 54.3     |
|    ep_rew_mean        | 54.3     |
| time/                 |          |
|    fps                | 561      |
|    

KeyboardInterrupt: 